In [8]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [9]:
from tqdm.notebook import tqdm

In [10]:
from azure_cosmos_db import *

azure_cosmos = AzureCosmos()

azure_cosmos.list_containers()

[INFO] Cosmos client created


['eventBrite_links',
 'eventBrite_outputs',
 'eventBrite_events',
 'dice_events',
 'shotgun_events']

# Events, Dice & Shotgun Scraper

In [11]:
SCRAPER_NAME = "eventBrite_links"
INPUT_FILE_PATH = "data/inputs/Internal Dating Batch 2 - Social Mixer.csv"
SHEET_NAME = ""

In [12]:
azure_cosmos.DATABASE_ID = "Scraper"
azure_cosmos.CONTAINER_NAME = SCRAPER_NAME

azure_cosmos.container = azure_cosmos.initialize_cosmosdb()

[INFO] Cosmos client created


In [13]:
df = pd.read_csv(INPUT_FILE_PATH)
SHEET_NAME = os.path.splitext(os.path.basename(INPUT_FILE_PATH))[0]
print(SHEET_NAME)
# df = pd.read_excel(INPUT_FILE_PATH, sheet_name=SHEET_NAME)

df.head()

Internal Dating Batch 2 - Social Mixer


,Links
0,https://www.eventbrite.com/d/united-states/pai...
1,https://www.eventbrite.com/d/united-states/pai...
2,https://www.eventbrite.com/d/united-states/pai...
3,https://www.eventbrite.com/d/united-states/pai...
4,https://www.eventbrite.com/d/united-states/pai...


In [14]:
urls = df["Links"].to_list()

for url in tqdm(urls, desc=f"Uploading {len(urls)} URLs to {SCRAPER_NAME}"):
        print(url)
        secure_url = url.replace("http://", "https://")
        data = {
            "id": hashlib.sha256(secure_url.encode()).hexdigest(),
            "url": secure_url,
            "processed": False,
            "sheet_name": SHEET_NAME
        }
        azure_cosmos.create_conversation(conversation_data=data)

Uploading 256 URLs to eventBrite_links:   0%|          | 0/256 [00:00<?, ?it/s]

https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=1
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=2
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=3
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=4
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=5
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=6
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=7
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid--parties/social-mixer/?page=8
[INFO] Conversation created successfully
https://www.eventbrite.com/d/united-states/paid-